In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from pathlib import Path
import csv
import re

In this project my fitness progress over time will be analyzed.
This is a bit tricky, since the data is stored in two different databases of two different apps (Progression and Workout Book?)

1. Loading the data
   1. Loading the gym app data
   2. Loading weight data



# 1. Loading the data
## 1.1 Loading the gym app data

CSV stands for comma separated values and is a file format, which is often used to represent data in a table structured form.
However, since a comma is seen as a separator between two columns, a comma inside one cell leads to a wrong parsing.
This can be fixed by replacing the comma with another value, e.g. a semicolon.

Example of the problem

![csv comma problem](res\csv_comma_problem.png)

The other problem was, that sometimes the comments contained a newline.

![csv line problem](res\csv_line_problem.png)

In [ ]:
def fix_progression_csv(input_file_path:Path, output_file_path=None) -> Path:
    """Makes the progression csv readable and returns the path to the fixed csv"""
    with open(input_file_path, "r") as input_file:
        reader = csv.reader(input_file)
        first_fix = fix_linebreaks(reader)
        second_fix = replace_comma_in_comments(first_fix)

    if not output_file_path:
        output_file_path = f"{input_file_path.stem}_fixed.csv"
    with open(output_file_path, "w") as output_file:
        writer = csv.writer(output_file, lineterminator="\n")
        [writer.writerow(row) for row in second_fix]
    return output_file_path


def fix_linebreaks(reader: csv.reader):
    """Revert line breaks and move them into one row.

    When writing a comment with a line break in the progression app, the part after the line break
    is written to a new line in the .csv file. This function detects all line breaks and replaces
    the line break with '.  '"""
    fixed_lines = [next(reader)]
    for line in reader:
        # Line break with empty line
        if not line:
            next_line = next(reader)
            last_line = fixed_lines[-1]
            fixed_lines[-1] = last_line[:-1] + [". ".join([last_line[-1], next_line[0]])] + next_line[1:]
        # Line break
        elif not re.match("\d{4}-\d{2}-\d{2}", line[0]):
            last_line = fixed_lines[-1]
            fixed_lines[-1] = last_line[:-1] + [". ".join([last_line[-1], line[0]])] + line[1:]
        else:
            fixed_lines.append(line)

    return iter(fixed_lines)


def replace_comma_in_comments(rows) -> list[list]:
    """Replace ',' in comments with ';'"2022-04-18 20 04 14.csv"

    When writing a comment with a ',' in the progression app, the .csv file treats it as new entry.
    This leads to a faulty parsing of the lines.
    To fix this, the comma is replaced with ';'"""
    header_line = next(rows)
    column_count = ",".join(header_line).count(",")
    fixed_lines = [header_line]

    for i, line in enumerate(rows):
        # Convert from list to string
        line = ",".join(line)
        comma_count = line.count(",")
        # Lines where a comma was written in either set or session comment
        if comma_count > column_count:
            first_parts = line.split(",")[:14]
            comment_parts = line.split(",")[14:-2]
            end_parts = line.split(",")[-2:]

            fixed_parts = [comment_parts[0]]
            for part in comment_parts[1:]:
                if part and part[0] == " ":
                    fixed_parts[-1] += ";" + part
                elif part and part[0].isdigit() and fixed_parts[-1][-1].isdigit():
                    fixed_parts[-1] += "." + part
                else:
                    fixed_parts.append(part)

            if len(fixed_parts) > 2:
                raise ValueError(f"Too many commas in line {i}: {line}")

            line = ",".join(first_parts + fixed_parts + end_parts)

        # Convert string back to list
        line = line.split(",")
        fixed_lines.append(line)

    return fixed_lines

In [ ]:
def read_csv(filepath: str, name: str) -> pd.DataFrame:
    """Read a csv file as dataframe."""
    if name == GYMBOOK_NAME:
        df = pd.read_csv(filepath, delimiter=";", decimal=",")
    else:
        df = pd.read_csv(filepath, delimiter=",", decimal=",")
    return df

# Data file of Progression app (android)
PROGRESSION_NAME = "progression_data"
GYMBOOK_NAME = "gymbook_data"
file1 = Path('data/2023-04-27 18 58 40.csv')
file1_fixed = fix_progression_csv(file1)
df_progression = read_csv(file1_fixed, PROGRESSION_NAME)

# Data file of GymBook app (iOS)
# Note: sep=, has to be added as first line in the csv file
# Also file has to be saved as utf-8 csv in Excel
file2 = Path('data/GymBook-Logs-2023-04-08.csv')
df_gymbook = read_csv(file2, GYMBOOK_NAME)

Let's take a closer look at the two loaded dataframes.

The progression dataframe ADD TEXT....

In [ ]:
df_progression.head(2)

In [ ]:
df_gymbook.head(2)

## 1.2 Loading the weight data

In [ ]:
df_weight = pd.read_csv("data/weight.csv", delimiter=";")
df_weight["Date"] = pd.to_datetime(df_weight["Date"], format="%Y-%m-%d")

df_weight.head(2)
# px.scatter(df, x="Date", y="Weight")

In [ ]:
# start_date = df_weight["Date"].iloc[-1]
# end_date = df_weight["Date"].iloc[0]
# date_range = pd.date_range(start_date, end_date)[::-1]

# dic = {"Date": date_range, "Weight": np.empty(len(date_range))}
# df_new = pd.DataFrame(dic)
# merge = pd.merge(df_weight, df_new, how="right", on="Date")
# merge["Weight_x"].update("Weight_y")
# merge.rename(columns={'Weight_x': 'Weight'}, inplace=True)
# merge.drop("Weight_y", axis=1, inplace=True)
# merge["Weight"].at[0] = 70.5
# merge["Weight"] = merge["Weight"].astype(float)
# px.scatter(x=merge["Date"], y=merge["Weight"].interpolate(method="polynomial", order=1))

## Data Cleanup
### Remove unwanted columns

The gymbook app also wrote a line when the exercise was not performed.
To get valid values those rows have to be removed.

In [ ]:
def remove_skipped_sets(df: pd.DataFrame) -> pd.DataFrame:
    """Remove sets from dataframe, which were not performed"""
    skipped_sets = df[df["Ausgelassen"] == "Ja"]
    df_no_skipped_sets = df.drop(skipped_sets.index, errors="ignore")
    return df_no_skipped_sets

df_gymbook = remove_skipped_sets(df_gymbook)

In [ ]:
def remove_redundant_columns(
    df: pd.DataFrame, unneeded_columns: list = None
) -> pd.DataFrame:
    """Remove columns which do not hold any valuable information."""
    constant_columns = [col for col in df.columns if df[col].nunique() <= 1]
    redundant_columns = constant_columns + unneeded_columns

    df_compact = df.drop(redundant_columns, axis=1, errors="ignore")
    print(f"Removed columns {redundant_columns}")
    return df_compact


df_gymbook = remove_redundant_columns(
    df_gymbook,
    [
        "Muskelgruppen (Primäre)",
        "Muskelgruppen (Sekundäre)",
        "Satz / Aufwärmsatz / Abkühlungssatz",
    ],
)
# Before removing Set Duration, write the info in the repetitions columns, used for time-based exercises e.g. Plank
df_progression["Repetitions"].fillna(df_progression["Set Duration (s)"], inplace=True)
df_progression = remove_redundant_columns(df_progression, ["Time", "Set Duration (s)"])

# Merge data

The columns of the gymbook app are in german, whereas the columns of the progression app are in english.
Some columns represent the same and must just be renamed.


In [ ]:
def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Rename gymbook columns to match naming of progression app"""
    column_name_mapping = {
        "Datum": "Date",
        "Training": "Workout Name",
        "Zeit": "Set Timestamp",
        "Übung": "Exercise Name",
        "Wiederholungen / Zeit": "Repetitions",
        "Gewicht / Strecke": "Weight",
        "Notizen": "Set Comment",
    }
    df_renamed_cols = df.rename(columns=column_name_mapping)
    return df_renamed_cols

def convert_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert number columns to int / float"""
    # Convert repetitions column from string to int
    df["Repetitions"] = df["Repetitions"].str.extract("(\d+)").astype(int)
    # Remove "kg" from weight column and convert from string to float
    df["Weight"] = df["Weight"].str.extract("(\d+,\d+)").replace(",", ".", regex=True).astype(float)
    return df

def adapt_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Rename and convert gymbook columns to match progression columns"""
    df = rename_columns(df)
    df = convert_columns(df)
    return df

df_gymbook = adapt_columns(df_gymbook)

In [ ]:
df_gymbook.head(2)

In [ ]:
df_progression.head(2)

Right now there are two different columns for the time. One date column and one set column. It is better to have the information in one.

In [ ]:
def merge_date_time_columns(
    df: pd.DataFrame,
    format_string: str,
    date_col: str = "Date",
    time_col: str = "Set Timestamp",
):
    """Replace date and time column by one datetime column."""
    df["Time"] = pd.to_datetime(df[date_col] + " " + df[time_col], format=format_string)
    df_datetime = df.drop([date_col, time_col], axis=1)
    return df_datetime


# Convert and combine date / time columns into one datetime column
df_progression = merge_date_time_columns(df_progression, "%Y-%m-%d %H:%M:%S")
df_gymbook = merge_date_time_columns(df_gymbook, "%d.%m.%Y %H:%M")

In [ ]:
df_progression.head(2)

In [ ]:
df_gymbook.head(2)

Now since both dataframes are cleansed, it is time to check which unique information each dataframe holds.
And if possible, to calculate the same information for the other dataframe

In [ ]:
# Check which columns are unique for each df
cols_only_in_progression = set(df_progression.columns).difference(
    set(df_gymbook.columns)
)
cols_only_in_gymbook = set(df_gymbook.columns).difference(set(df_progression.columns))
print(f"Columns only in progression: {cols_only_in_progression}")
print(f"Columns only in gymbook: {cols_only_in_gymbook}")


def extend_gymbook_df(df: pd.DataFrame) -> pd.DataFrame:
    """Add Session Duration and Set Order column to gymbook df, to match progression df"""
    # Calculate workout time using time difference between first and last set for each day
    df["Session Duration (s)"] = (
        df.groupby(df["Time"].dt.date)["Time"]
        .transform(lambda x: x.max() - x.min())
        .dt.seconds
    )
    df["Set Order"] = df.groupby([df["Time"].dt.date, "Exercise Name"]).cumcount() + 1
    return df

df_gymbook = extend_gymbook_df(df_gymbook)

Now both dataframes can be merged.
It also checked, if all columns have the correct dtype.

In [ ]:
df = pd.concat([df_progression, df_gymbook])
df = df.sort_values("Time", ascending=False)

# Remove NaNs
df["Weight"] = df["Weight"].fillna(0).astype(float)

Last step is to find the same exercises with different names. E.g. in the gymbook app the squat is called "Barbell Squats" and in the gymbook app it is called "Barbell Squat"

Gymbook uses plural instead of a singular for exercise names. Convert to singular and check again

In [ ]:
exercises_progression = set(df_progression["Exercise Name"])
exercises_gymbook = set(df_gymbook["Exercise Name"])
common_exercises = exercises_gymbook.intersection(exercises_progression)

print(f"{len(common_exercises)} exercises are in both: {common_exercises}")
exercises_gymbook_unique = exercises_gymbook.symmetric_difference(common_exercises)
print(f"{len(exercises_gymbook_unique)} Exercises are only in gymbook: {exercises_gymbook_unique}")
exercises_progression_unique = exercises_progression.symmetric_difference(common_exercises)
print(f"{len(exercises_progression_unique)} Exercises are only in progression: {exercises_progression_unique}")


def singularize(exercises: set[str]) -> dict:
    """Convert from plural form to singular form"""
    # Plural form es but keep e
    plural_exceptions = ["lunges", "raises"]
    singles = []
    for exercise in exercises:
        if exercise[-2:] == "es":
            singular_replacement = exercise[:-2]
            singles.append([exercise, singular_replacement])
        elif exercise[-1:] == "s" and not exercise[-2:] == "ss":
            singular_replacement = exercise[:-1]
            singles.append([exercise, singular_replacement])

    map_plural_to_singular = dict(singles)
    plural_exceptions = {"Lunges", "Raises"}
    for exercise in map_plural_to_singular:
        for exception in plural_exceptions:
            if exception in exercise:
                singular_replacement = exercise.replace(exception[:-2], exception[:-1])
                map_plural_to_singular[exercise] = singular_replacement
    return map_plural_to_singular


map_gymbook_plural_to_singular = singularize(set(df_gymbook["Exercise Name"]))

# Convert some gymbook names to progression names
map_gymbook_to_progression = {
    "Ab Wheel": "Ab Roller",
    "Alternating Dumbbell Preacher Curl": "Alternating Dumbbell Curl",
    "Arnold Press": "Arnold Dumbbell Press (Seated)",
    "Back Extension": "Machine Hyperextension",
    "Bulgarian Split Squat ": "Bulgarian Split Squat",
    "Cable Fly": "Cable Back Fly",
    "Calf Press in Leg Press": "Machine Calf Press",
    "Chin-Up": "Chinup",
    "Concentration Curl": "Dumbbell Concentration Curl",
    "Crunch": "Weighted Crunch",
    "Decline Push-Up": "Decline Pushup",
    "Dumbbell Lateral Raise": "Dumbbell Side Raise",
    "Dumbbell Press": "Dumbbell Shoulder Press",
    "Dumbbell Row": "Bent-Over Dumbbell Row",
    "Dumbbell Skullcrusher": "Lying Dumbbell Skull Crusher",
    "Hammer Curl": "Dumbbell Hammer Curl",
    "Kneeling Cable Crunch": "Cable Crunch",
    "Lat Pull-Down": "Machine Lat Pulldown",
    "Leg Extension": "Machine Leg Extension",
    "Leg Press": "Machine Leg Press",
    "Low Cable One-Arm Lateral Raise": "Cable Side Raise",
    "Lying Dumbbell Triceps Extension": "Dumbbell Triceps Extension",
    "Lying EZ-Bar Triceps Extension": "Lying Barbell Skull Crusher",
    "Lying Leg Curl": "Machine Lying Leg Curl",
    "Machine Back Extension": "Machine Hyperextension",
    "Machine Hip Abduction": "Machine Thigh Abduction (Out)",
    "Machine Trunk Rotation": "Torso Rotation Machine",
    "One-Leg Leg Extension": "Machine Single-Leg Extension",
    "Power Clean": "Barbell Power Clean",
    "Pullups Weighted ": "Weighted Pullup",
    "Push Down": "Cable Pushdown (with Bar Handle)",
    "Push Press": "Barbell Push Press",
    "Push-Up": "Pushup",
    "Seated Leg Curl": "Machine Leg Curl",
    "Seated Machine Hip Abduction": "Machine Thigh Abduction (Out)",
    "Seated Machine Row": "Machine Row",
    "Standing Calf Raise": "Machine Calf Raise",
    "Standing Machine Calf Raise": "Machine Calf Raise",
    "Wide-Grip Lat Pull-Down": "Wide-Grip Machine Lat Pulldown",
}

# Convert some progression names to gymbook names
map_progression_to_gymbook = {
    "Barbell Curl": "EZ-Bar Curl",
    "Bent-Over Barbell Row": "Barbell Row",
    "Butterfly Reverse": "Reverse Machine Fly",
    "Cable Row": "Seated Cable Row",
    "Dumbbell Pullover (Targeting back)": "Dumbbell Lat Pullover",
    "Farmer's Walk (with Dumbbells)": "Farmers Walk",
    "Farmer's Walk (with Weight Plate)": "Farmers Walk",
    "Machine Calf Press": "Calf Press In Leg Press",
    "Press around": "Press Around",
    "Romanian Deadlift": "Barbell Romanian Deadlift",
    "Stiff-Leg Deadlift (Wide Stance)": "Straight-Leg Barbell Deadlift",
    "Sumo Deadlift": "Barbell Sumo Deadlift",
    "Weighted pistol squat": "Pistol squat",
}

# Convert some names to a value, which is in neither dataframe
map_rename_in_both = {
    # Gymbook
    "Close-Grip Lat Pull-Down": "Close-Grip Machine Lat Pull-Down",
    "Machine Bench Press": "Seated Machine Bench Press",
    "Parallel Bar Dip": "Dip",
    # Progression
    "Barbell Shrug (Behind the Back)": "Barbell Shrug",
    "Chest Dip": "Dip",
    "Machine Bench Press": "Seated Machine Bench Press",
}

print(f"At the beginning there are {df['Exercise Name'].nunique()} unique exercises")

df["Exercise Name"].replace(map_gymbook_plural_to_singular, inplace=True)
print(f"After singularizing there are {df['Exercise Name'].nunique()} unique exercises")

df["Exercise Name"].replace(map_gymbook_to_progression, inplace=True)
print(f"After mapping gymbook exercises to progression naming there are {df['Exercise Name'].nunique()} unique exercises")

df["Exercise Name"].replace(map_progression_to_gymbook, inplace=True)
print(f"After mapping progression exercises to gymbook naming there are {df['Exercise Name'].nunique()} unique exercises")

df["Exercise Name"].replace(map_rename_in_both, inplace=True)
print(f"After renaming exercises in both there are {df['Exercise Name'].nunique()} unique exercises")

In [ ]:
# Correct dtypes
print(df.dtypes)
df["Workout Name"] = df["Workout Name"].astype("category")
df["Exercise Name"] = df["Exercise Name"].astype("category")

In [ ]:
# Convert from german to english
map_muscle_category_ger_eng = {
    "Arme": "Arms",
    "Bauch": "Abs",
    "Beine": "Legs",
    "Brust": "Chest",
    "Gesäss": "Glute",
    "Rücken": "Back",
    "Schultern": "Shoulders",
}
df["Bereich"].replace(map_muscle_category_ger_eng, inplace=True)
df["Bereich"].replace(np.nan, "Undefined", inplace=True)

exercises_without_category = set(df[df["Bereich"] == "Undefined"]["Exercise Name"])
exercises_with_category = set(df[df["Bereich"] != "Undefined"]["Exercise Name"])
# Exercises which where mapped from gymbook to progression naming and did loose their Bereich
exercises_with_lost_category = exercises_with_category.intersection(exercises_without_category)
print(f"{len(exercises_without_category)} have no category")
print(f"{len(exercises_with_category)} have a category")
print(f"{len(exercises_with_lost_category)} have only partially category mapping")

for exercise in exercises_with_lost_category:
    muscle_category = next(iter(set(df[df["Exercise Name"] == exercise]["Bereich"]) - {"Undefined"}))
    df.loc[(df["Exercise Name"] == exercise) & (df["Bereich"] == "Undefined"), "Bereich"] = muscle_category
    
exercises_without_category = set(df[df["Bereich"] == "Undefined"]["Exercise Name"])
exercises_with_category = set(df[df["Bereich"] != "Undefined"]["Exercise Name"])
exercises_with_lost_category = exercises_with_category.intersection(exercises_without_category)
print(f"{len(exercises_without_category)} have no category")
print(f"{len(exercises_with_category)} have a category")
print(f"{len(exercises_with_lost_category)} have only partially category mapping")


In [ ]:
map_exercise_to_muscle = {
    "Arms": {"Push", "Curl", "Kickback", "Triceps"},
    "Back": {"Pull", "Row", "deadlift"},
    "Chest": {"Bench", "Crossover", "Fly"},
    "Legs": {"Calf", "Leg", "Squat", "Lunge", "Thigh", "Clean"},
    "Shoulders": {"Shoulder", "Shrug", "Delt", "Arnold", "Raise"},
}

for exercise in exercises_without_category:
    counter = 0
    muscle_categories = []
    for category, keywords in map_exercise_to_muscle.items():
        for keyword in keywords:
            if keyword in exercise:
                muscle_category = category
                muscle_categories.append(category)
                counter += 1
    if counter == 1:
        # print(f"Mapping {exercise} to {muscle_category}")
        df.loc[df["Exercise Name"] == exercise, "Bereich"] = muscle_category
    elif counter > 1:
        # Legs always wins
        if "Legs" in muscle_categories:
            df.loc[df["Exercise Name"] == exercise, "Bereich"] = "Legs"
        else:
            print(f"Problem with {exercise}. Found in {muscle_categories}. Skipping...")
    else:
        pass
        print(f"No mapping for {exercise}")

# Solving conflicts
map_conflicts = {
    "Shoulders": {"Cable Rear Delt Fly"},
    "Back": {"Single-Arm Dumbbell Row on Bench"},
    "Chest": {"Barbell Bench Press (with Raised Feet)"}
}
# Map the rest manually
manual_map = {
    "Abs": {
        "Ab Complex",
        "Bicycle Crunch",
        "Burpee",
        "Plank",
        "Russian Twist",
        "Standing Cable Lift",
    },
    "Back": {"Weighted Chinup"},
    "Arms": {"Wrist curl (Roller)"},
}
# map_conflicts.update(manual_map)

for muscle_category, exercises in map_conflicts.items():
    for exercise in exercises:
        df.loc[df["Exercise Name"] == exercise, "Bereich"] = muscle_category
        
for muscle_category, exercises in manual_map.items():
    for exercise in exercises:
        df.loc[df["Exercise Name"] == exercise, "Bereich"] = muscle_category

In [ ]:
exercises_without_category = set(df[df["Bereich"] == "Undefined"]["Exercise Name"])
exercises_without_category_l = sorted(list(exercises_without_category))
# exercises_without_category

In [ ]:
df["Weight"].unique()

In [ ]:
weekday_map = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Weekday"] = df["Time"].dt.weekday.map(weekday_map)

In [ ]:
df.head()

In [ ]:
df.groupby([df["Time"].dt.year, df["Time"].dt.month, df["Time"].dt.day]).nunique()

In [ ]:
len(df.groupby([df["Time"].dt.year, df["Time"].dt.month, df["Time"].dt.date]).nunique().loc[(2022, 7)])

In [ ]:
df_dates = df.groupby([df["Time"].dt.year.rename("year"), df["Time"].dt.month.rename("month")])["Time"].apply(lambda x: x.dt.date.nunique())
df_dates = df_dates.reset_index(level=[0,1])
df_dates["datetime"] = pd.to_datetime(df_dates[["year", "month"]].assign(day=1))

In [ ]:
exercise = "Barbell Squat"
df_filtered_exercise = df[df["Exercise Name"] == exercise]

In [ ]:
grouped_by_date = df_filtered_exercise.groupby(df_filtered_exercise["Time"].dt.date)
def calculate_volumne(group):
    return (group["Repetitions"] * group["Weight"]).sum()

volumne = grouped_by_date.apply(calculate_volumne)
# px.scatter(volumne,)
df_v = pd.DataFrame(volumne)
df_v.isna().sum()
df_v

In [ ]:
df["Time"][0]

In [ ]:
from datetime import datetime
date_string=  '2023-09-01'
datetime.strptime(date_string, "%Y-%m-%d")

In [ ]:
grouped_by_date = df_filtered_exercise.groupby(df_filtered_exercise["Time"].dt.date)
weekdays = pd.Categorical(grouped_by_date["Weekday"].first(), categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
weekdays.sort_values()
px.histogram(weekdays.sort_values())
# px.bar(weekdays)